## 1. Data Collection

In [3]:
from selenium import webdriver
import json
import re
import pandas as pd
import os
import urllib.request
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import matplotlib.pyplot as plt
import cv2
import numpy as np
from IPython.core.display import HTML
import glob
import random
import base64
from PIL import Image
from io import BytesIO
import requests
import matplotlib.image as mpimg
from django.utils import encoding
import pickle
from lxml import html
from fake_useragent import UserAgent
ua = UserAgent()
from selenium import webdriver
import bs4
import random
from itertools import cycle
from selenium.common.exceptions import NoSuchElementException

### 1. News data from New York Times

#### 1-1. Write function to scrap news data from website:

For text analytics to be performed, collect news category, date of release, headline, one line summary from NY Times website. I have used selenium webdriver and xpath to scrap the data from period 2010-01-01 to 2019-04-30.

In [6]:
# Open NYT webpage by using Selenium Webdriver
# get the path of ChromeDriverServer and open browser
chrome_driver_path = r"C:\Users\Natalie Kim\Documents\Python Scripts\chromedriver_win32_74\chromedriver.exe"
browser = webdriver.Chrome(chrome_driver_path)

In [869]:
def get_news(start,end):
    """
    
    collect news category, headline, one-line summary and date of release into pandas Dataframe
    Input period start and end date
    
    """
    # step1: open nytimes apple page using pre-determined ua header
    # available user agent found at https://udger.com/resources/ua-list/browser-detail?browser=Chrome
    page = 'https://www.nytimes.com/search?endDate='+end+'&query=apple&sort=best&startDate='+start
    browser.get(page)
    headers = {'User-Agent':'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/9.0.600.0 Safari/534.14'}
    r = requests.get(page,headers=headers)
    
    # step2: Extend to the end of of news list by clicking "show more" repetitively
    stop = 0
    while stop == 0:
        webElem = browser.find_element_by_xpath("//*/button[@type = 'button']")
        if stop == 0:
            webElem.click()
            time.sleep(7)
            try:
                webElem = browser.find_element_by_xpath("//*/button[@type = 'button']")
            except NoSuchElementException:
                stop = 1
                
    # step3: obtain date, headline, content, category by xpath
    doc = html.fromstring(browser.page_source)
    xpath_date = '//*/li[@data-testid="search-bodega-result"]/div/time/text()'
    xpath_headline = '//*/li[@data-testid="search-bodega-result"]/div/div/a/h4/text()'
    xpath_no_p = '//*/li[@data-testid="search-bodega-result"]/div/div/a[not(p)]'
    x_path_all = '//*/li[@data-testid="search-bodega-result"]/div/div/a'
    
    dates = doc.xpath(xpath_date)    
    contents = [x.text for x in doc.xpath('//*/li[@data-testid="search-bodega-result"]/div/div/a/p[position()<2]')]
    categories = [x.text for x in doc.xpath('//*/li[@data-testid="search-bodega-result"]/div/div/p')]
    headlines = doc.xpath(xpath_headline)
    all_path = doc.xpath(x_path_all)
    no_p = doc.xpath(xpath_no_p)
    to_exclude_index = [i for i in range(len(all_path)) if all_path[i] in no_p]
    
    dates = [dates[i] for i in range(len(dates)) if i not in to_exclude_index]
    headlines = [headlines[i] for i in range(len(headlines)) if i not in to_exclude_index]
    categories = [categories[i] for i in range(len(categories)) if i not in to_exclude_index]
    Merged_content = [str(i) +', '+ str(j) for i, j in zip(headlines, contents)] 
    
    # step4: convert it to dictionary and save/return Dataframe
    data={
    'DATE': dates,
    'CATEGORIES': categories,
    'TITLE': headlines,
    'CONTENT': contents,
    'ALL_CONTENT':Merged_content
    }
    df = pd.DataFrame(data)
    
    return df

In [765]:
def save_as_csv(df,filename):
    """
    save dataframe collected to csv file with filename provided as input
    """
    df.to_csv(filename+'.txt', sep='\t', encoding='utf-8')

#### 1-2. collect data by selenium webdriver:

I have divided period into sub period (six-month) for purpose of checking progress of data collection and ease of debugging. After collecting all data, merge it into one dataframe for the ease of analysis later.

In [766]:
h1_2019 = get_news('20190101','20190430')
h1_2019.head()

,DATE,CATEGORIES,TITLE,CONTENT,ALL_CONTENT
0,May 1,Technology,Apple’s Plan to Buy $75 Billion of Its Stock F...,The company said it would add to its record re...,Apple’s Plan to Buy $75 Billion of Its Stock F...
1,April 7,Business,"Scheme to Swap Fake iPhones Adds Up to $900,00...",Two college students in Oregon are accused of ...,"Scheme to Swap Fake iPhones Adds Up to $900,00..."
2,April 4,Opinion,The Incredible Shrinking Apple,Steve Jobs wanted to put a ding in the univers...,"The Incredible Shrinking Apple, Steve Jobs wan..."
3,April 3,Personal Tech,"Apple AirPods Review: Perfect Earbuds, but The...","Sleek and convenient, Apple’s wireless earbuds...","Apple AirPods Review: Perfect Earbuds, but The..."
4,March 27,Media,Hollywood Had Questions. Apple Didn’t Answer T...,The event at the Steve Jobs Theater was heavy ...,Hollywood Had Questions. Apple Didn’t Answer T...


In [931]:
new_date = [date +", 2019" for date in h1_2019['DATE']]
h1_2019['DATE'] = new_date

In [1022]:
h1_2019.head()

,DATE,CATEGORIES,TITLE,CONTENT,ALL_CONTENT
0,"May 1, 2019",Technology,Apple’s Plan to Buy $75 Billion of Its Stock Fuels Spending Debate,"The company said it would add to its record returns to shareholders, most likely fueling an intensifying debate over corporate spending.","Apple’s Plan to Buy $75 Billion of Its Stock Fuels Spending Debate, The company said it would add to its record returns to shareholders, most likely fueling an intensifying debate over corporate spending."
1,"April 7, 2019",Business,"Scheme to Swap Fake iPhones Adds Up to $900,000 Loss for Apple, Prosecutors Say","Two college students in Oregon are accused of tricking Apple into replacing nearly 1,500 counterfeit iPhones with genuine ones that were later sold.","Scheme to Swap Fake iPhones Adds Up to $900,000 Loss for Apple, Prosecutors Say, Two college students in Oregon are accused of tricking Apple into replacing nearly 1,500 counterfeit iPhones with genuine ones that were later sold."
2,"April 4, 2019",Opinion,The Incredible Shrinking Apple,"Steve Jobs wanted to put a ding in the universe. Today, Apple wants to ding your pocketbook.","The Incredible Shrinking Apple, Steve Jobs wanted to put a ding in the universe. Today, Apple wants to ding your pocketbook."
3,"April 3, 2019",Personal Tech,"Apple AirPods Review: Perfect Earbuds, but They Don’t Last","Sleek and convenient, Apple’s wireless earbuds are an engineering marvel. One problem: They are difficult and expensive to maintain.","Apple AirPods Review: Perfect Earbuds, but They Don’t Last, Sleek and convenient, Apple’s wireless earbuds are an engineering marvel. One problem: They are difficult and expensive to maintain."
4,"March 27, 2019",Media,Hollywood Had Questions. Apple Didn’t Answer Them.,The event at the Steve Jobs Theater was heavy on star power and light on specifics. The Los Angeles crowd buzzed more about a new credit card than anything else.,"Hollywood Had Questions. Apple Didn’t Answer Them., The event at the Steve Jobs Theater was heavy on star power and light on specifics. The Los Angeles crowd buzzed more about a new credit card than anything else."


In [767]:
save_as_csv(h1_2019,'h1_2019')

In [768]:
h2_2018 = get_news('20180701','20181231')
save_as_csv(h2_2018,'h2_2018')

In [963]:
new_date_2018 = [date +", 2018" if date == 'Jan. 1' else date for date in h2_2018['DATE'] ]
h2_2018['DATE'] = new_date_2018

In [ ]:
h1_2018 = get_news('20180101','20180630')
save_as_csv(h1_2018,'h1_2018')

In [797]:
h2_2017 = get_news('20170701','20171231')
save_as_csv(h2_2017,'h2_2017')

In [ ]:
h1_2017 = get_news('20170101','20170630')
save_as_csv(h1_2017,'h1_2017')

In [839]:
h2_2016 = get_news('20160701','20161231')
save_as_csv(h2_2016,'h2_2016')

In [840]:
h1_2016 = get_news('20160101','20160630')
save_as_csv(h1_2016,'h1_2016')

In [841]:
h2_2015 = get_news('20150701','20151231')
save_as_csv(h2_2015,'h2_2015')

In [842]:
h1_2015 = get_news('20150101','20150630')
save_as_csv(h1_2015,'h1_2015')

In [843]:
h2_2014 = get_news('20140701','20141231')
save_as_csv(h2_2014,'h2_2014')

In [ ]:
h1_2014 = get_news('20140101','20140630')
save_as_csv(h1_2014,'h1_2014')

In [870]:
h2_2013 = get_news('20130701','20131231')
save_as_csv(h2_2013,'h2_2013')

In [871]:
h1_2013 = get_news('20130101','20130630')
save_as_csv(h1_2013,'h1_2013')

In [872]:
h2_2012 = get_news('20120701','20121231')
save_as_csv(h2_2012,'h2_2012')

In [873]:
h1_2012 = get_news('20120101','20120630')
save_as_csv(h1_2012,'h1_2012')

In [874]:
h2_2011 = get_news('20110701','20111231')
save_as_csv(h2_2011,'h2_2011')

In [875]:
h1_2011 = get_news('20110101','20110630')
save_as_csv(h1_2011,'h1_2011')

In [877]:
h2_2010 = get_news('20100701','20101231')
save_as_csv(h2_2010,'h2_2010')

In [878]:
h1_2010 = get_news('20100101','20100630')
save_as_csv(h1_2010,'h1_2010')

In [926]:
h1_2010.head()

,DATE,CATEGORIES,TITLE,CONTENT,ALL_CONTENT
0,"June 29, 2010",Technology,Apple Sells 1.7 Million New iPhones in Three Days,Apple said the release of the iPhone 4 was its most successful product introduction yet.,"Apple Sells 1.7 Million New iPhones in Three Days, Apple said the release of the iPhone 4 was its most successful product introduction yet."
1,"June 24, 2010",Technology,Is Apple a Victim of Sour Grapes?,"Once dismissed by larger companies, Apple is now viewed by some as an anticompetitive industry bully.","Is Apple a Victim of Sour Grapes?, Once dismissed by larger companies, Apple is now viewed by some as an anticompetitive industry bully."
2,"June 29, 2010",Technology,Germany Asks Apple About iPhone’s Data-Gathering,"Like Google and Facebook, Apple has been confronted with Germany’s privacy laws, among Europe’s stiffest.","Germany Asks Apple About iPhone’s Data-Gathering, Like Google and Facebook, Apple has been confronted with Germany’s privacy laws, among Europe’s stiffest."
3,"June 24, 2010",Technology,Class Action Against Apple and AT&T Is Amended,A lawsuit filed against AT&T and Apple over the elimination of the iPad 3G unlimited data plan was amended and refiled as a class action lawsuit on Wednesday in a California district court.,"Class Action Against Apple and AT&T Is Amended, A lawsuit filed against AT&T and Apple over the elimination of the iPad 3G unlimited data plan was amended and refiled as a class action lawsuit on Wednesday in a California district court."
4,"June 23, 2010",Personal Tech,"New iPhone Arrives; Rivals, Beware","The iPhone 4 is slimmer, with an improved screen and audio. But it was a hit before the public knew that.","New iPhone Arrives; Rivals, Beware, The iPhone 4 is slimmer, with an improved screen and audio. But it was a hit before the public knew that."


#### 1-3. Data cleaning:

Format date colume into DD/MM/YYYY for consistency and further analysis. In terms of content, I have decided to only include: ['Technology','Business','Business Computing','Companies','Economy','Global Business','International Business','Internet','Personal Tech','Stocks and Bonds']
categories to only reflect news that are relevant to Apple stock.

Firstly, merge all dataframe into one.

In [992]:
# merge all dataframe into one
pdList = [h1_2019, h1_2018, h2_2018,h1_2017, h2_2017,h1_2016, h2_2016,h1_2015, h2_2015,
          h1_2014, h2_2014,h1_2013, h2_2013,h1_2012, h2_2012,h1_2011, h2_2011,h1_2010, h2_2010]
merged_df = pd.concat(pdList)
save_as_csv(merged_df,'merged_all_period') #for backup

C:\Users\Natalie Kim\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [1021]:
new_merged_df = merged_df[merged_df.DATE != 'Feb. 14']
new_merged_df.head()

,ALL_CONTENT,CATEGORIES,CONTENT,DATE,TITLE
0,"Apple’s Plan to Buy $75 Billion of Its Stock Fuels Spending Debate, The company said it would add to its record returns to shareholders, most likely fueling an intensifying debate over corporate spending.",Technology,"The company said it would add to its record returns to shareholders, most likely fueling an intensifying debate over corporate spending.","May 1, 2019",Apple’s Plan to Buy $75 Billion of Its Stock Fuels Spending Debate
1,"Scheme to Swap Fake iPhones Adds Up to $900,000 Loss for Apple, Prosecutors Say, Two college students in Oregon are accused of tricking Apple into replacing nearly 1,500 counterfeit iPhones with genuine ones that were later sold.",Business,"Two college students in Oregon are accused of tricking Apple into replacing nearly 1,500 counterfeit iPhones with genuine ones that were later sold.","April 7, 2019","Scheme to Swap Fake iPhones Adds Up to $900,000 Loss for Apple, Prosecutors Say"
2,"The Incredible Shrinking Apple, Steve Jobs wanted to put a ding in the universe. Today, Apple wants to ding your pocketbook.",Opinion,"Steve Jobs wanted to put a ding in the universe. Today, Apple wants to ding your pocketbook.","April 4, 2019",The Incredible Shrinking Apple
3,"Apple AirPods Review: Perfect Earbuds, but They Don’t Last, Sleek and convenient, Apple’s wireless earbuds are an engineering marvel. One problem: They are difficult and expensive to maintain.",Personal Tech,"Sleek and convenient, Apple’s wireless earbuds are an engineering marvel. One problem: They are difficult and expensive to maintain.","April 3, 2019","Apple AirPods Review: Perfect Earbuds, but They Don’t Last"
4,"Hollywood Had Questions. Apple Didn’t Answer Them., The event at the Steve Jobs Theater was heavy on star power and light on specifics. The Los Angeles crowd buzzed more about a new credit card than anything else.",Media,The event at the Steve Jobs Theater was heavy on star power and light on specifics. The Los Angeles crowd buzzed more about a new credit card than anything else.,"March 27, 2019",Hollywood Had Questions. Apple Didn’t Answer Them.


Second, select only relevant news category for analysis. As I will be analyzing on scope of apple's finanacials and macroeconomic/global economy,only include specific sub-categories among 161 categories.

In [12]:
#formatting
pd.set_option('display.max_colwidth', -1)

In [1011]:
len(set(new_merged_df['CATEGORIES']))

161

In [1012]:
selected_categories = ['Technology','Business','Business Computing','Companies','Economy','Global Business','International Business','Internet','Personal Tech','Stocks and Bonds']
sub_df = new_merged_df[new_merged_df['CATEGORIES'].isin(selected_categories)]
sub_df.head()

,ALL_CONTENT,CATEGORIES,CONTENT,DATE,TITLE
0,"Apple’s Plan to Buy $75 Billion of Its Stock Fuels Spending Debate, The company said it would add to its record returns to shareholders, most likely fueling an intensifying debate over corporate spending.",Technology,"The company said it would add to its record returns to shareholders, most likely fueling an intensifying debate over corporate spending.","May 1, 2019",Apple’s Plan to Buy $75 Billion of Its Stock Fuels Spending Debate
1,"Scheme to Swap Fake iPhones Adds Up to $900,000 Loss for Apple, Prosecutors Say, Two college students in Oregon are accused of tricking Apple into replacing nearly 1,500 counterfeit iPhones with genuine ones that were later sold.",Business,"Two college students in Oregon are accused of tricking Apple into replacing nearly 1,500 counterfeit iPhones with genuine ones that were later sold.","April 7, 2019","Scheme to Swap Fake iPhones Adds Up to $900,000 Loss for Apple, Prosecutors Say"
3,"Apple AirPods Review: Perfect Earbuds, but They Don’t Last, Sleek and convenient, Apple’s wireless earbuds are an engineering marvel. One problem: They are difficult and expensive to maintain.",Personal Tech,"Sleek and convenient, Apple’s wireless earbuds are an engineering marvel. One problem: They are difficult and expensive to maintain.","April 3, 2019","Apple AirPods Review: Perfect Earbuds, but They Don’t Last"
6,"Apple Unveils Video Service With Celebrity Parade but Few Details, The iPhone maker is partnering with Hollywood heavyweights and publishers to provide broad new services to its users.",Technology,The iPhone maker is partnering with Hollywood heavyweights and publishers to provide broad new services to its users.,"March 26, 2019",Apple Unveils Video Service With Celebrity Parade but Few Details
8,"Apple Cracks Down on Apps That Fight iPhone Addiction, Last year, with much fanfare, the tech giant unveiled a screen-time tracker of its own. Then it quietly began purging competitors from its store.",Technology,"Last year, with much fanfare, the tech giant unveiled a screen-time tracker of its own. Then it quietly began purging competitors from its store.","April 27, 2019",Apple Cracks Down on Apps That Fight iPhone Addiction


Third, sort out date formatting. Convert the current format to DD/MM/YYYY for consistency and ease of further analysis.

In [1013]:
converted_date = []
fail = []
exl_month = ['March', 'April', 'May', 'June', 'July']

for date in sub_df['DATE']:
    try:
        if any(month in date for month in exl_month):
            dateTime = datetime.datetime.strptime(date, "%B %d, %Y")
            converted_date.append(dateTime.strftime('%d/%m/%Y'))
        elif 'Sept' in date:
            new = date.replace("Sept", "Sep")
            dateTime = datetime.datetime.strptime(new, "%b. %d, %Y")
            converted_date.append(dateTime.strftime('%d/%m/%Y')) 
        else:
            dateTime = datetime.datetime.strptime(date, "%b. %d, %Y")
            converted_date.append(dateTime.strftime('%d/%m/%Y'))
    except:
        fail.append(date)

In [1017]:
# add new converted date into dataframe
sub_df['CONVERTED_DATE'] = converted_date

C:\Users\Natalie Kim\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [1019]:
save_as_csv(sub_df,'cleaned_final') #for backup
sub_df.head()

,ALL_CONTENT,CATEGORIES,CONTENT,DATE,TITLE,CONVERTED_DATE
0,"Apple’s Plan to Buy $75 Billion of Its Stock Fuels Spending Debate, The company said it would add to its record returns to shareholders, most likely fueling an intensifying debate over corporate spending.",Technology,"The company said it would add to its record returns to shareholders, most likely fueling an intensifying debate over corporate spending.","May 1, 2019",Apple’s Plan to Buy $75 Billion of Its Stock Fuels Spending Debate,01/05/2019
1,"Scheme to Swap Fake iPhones Adds Up to $900,000 Loss for Apple, Prosecutors Say, Two college students in Oregon are accused of tricking Apple into replacing nearly 1,500 counterfeit iPhones with genuine ones that were later sold.",Business,"Two college students in Oregon are accused of tricking Apple into replacing nearly 1,500 counterfeit iPhones with genuine ones that were later sold.","April 7, 2019","Scheme to Swap Fake iPhones Adds Up to $900,000 Loss for Apple, Prosecutors Say",07/04/2019
3,"Apple AirPods Review: Perfect Earbuds, but They Don’t Last, Sleek and convenient, Apple’s wireless earbuds are an engineering marvel. One problem: They are difficult and expensive to maintain.",Personal Tech,"Sleek and convenient, Apple’s wireless earbuds are an engineering marvel. One problem: They are difficult and expensive to maintain.","April 3, 2019","Apple AirPods Review: Perfect Earbuds, but They Don’t Last",03/04/2019
6,"Apple Unveils Video Service With Celebrity Parade but Few Details, The iPhone maker is partnering with Hollywood heavyweights and publishers to provide broad new services to its users.",Technology,The iPhone maker is partnering with Hollywood heavyweights and publishers to provide broad new services to its users.,"March 26, 2019",Apple Unveils Video Service With Celebrity Parade but Few Details,26/03/2019
8,"Apple Cracks Down on Apps That Fight iPhone Addiction, Last year, with much fanfare, the tech giant unveiled a screen-time tracker of its own. Then it quietly began purging competitors from its store.",Technology,"Last year, with much fanfare, the tech giant unveiled a screen-time tracker of its own. Then it quietly began purging competitors from its store.","April 27, 2019",Apple Cracks Down on Apps That Fight iPhone Addiction,27/04/2019


In [19]:
# to convert back to pd from txt file
import pandas as pd
import datetime

%cd C:\Users\Natalie Kim\Documents\Python Scripts
new_df = pd.read_csv('cleaned_final.txt',sep='\t')
new_df.drop(['Unnamed: 0'],axis=1,inplace = True)
converted_date =[]
new_df['DATE'] = new_df['CONVERTED_DATE']
for date in new_df['DATE']:
    dateTime = datetime.datetime.strptime(date, '%d/%m/%Y')
    converted_date.append(dateTime.strftime('%Y-%m-%d'))
new_df['DATE'] = converted_date
new_df.drop(['CONVERTED_DATE'],axis=1,inplace = True)
new_df.head()

C:\Users\Natalie Kim\Documents\Python Scripts


,ALL_CONTENT,CATEGORIES,CONTENT,DATE,TITLE
0,"Apple’s Plan to Buy $75 Billion of Its Stock Fuels Spending Debate, The company said it would add to its record returns to shareholders, most likely fueling an intensifying debate over corporate spending.",Technology,"The company said it would add to its record returns to shareholders, most likely fueling an intensifying debate over corporate spending.",2019-05-01,Apple’s Plan to Buy $75 Billion of Its Stock Fuels Spending Debate
1,"Scheme to Swap Fake iPhones Adds Up to $900,000 Loss for Apple, Prosecutors Say, Two college students in Oregon are accused of tricking Apple into replacing nearly 1,500 counterfeit iPhones with genuine ones that were later sold.",Business,"Two college students in Oregon are accused of tricking Apple into replacing nearly 1,500 counterfeit iPhones with genuine ones that were later sold.",2019-04-07,"Scheme to Swap Fake iPhones Adds Up to $900,000 Loss for Apple, Prosecutors Say"
2,"Apple AirPods Review: Perfect Earbuds, but They Don’t Last, Sleek and convenient, Apple’s wireless earbuds are an engineering marvel. One problem: They are difficult and expensive to maintain.",Personal Tech,"Sleek and convenient, Apple’s wireless earbuds are an engineering marvel. One problem: They are difficult and expensive to maintain.",2019-04-03,"Apple AirPods Review: Perfect Earbuds, but They Don’t Last"
3,"Apple Unveils Video Service With Celebrity Parade but Few Details, The iPhone maker is partnering with Hollywood heavyweights and publishers to provide broad new services to its users.",Technology,The iPhone maker is partnering with Hollywood heavyweights and publishers to provide broad new services to its users.,2019-03-26,Apple Unveils Video Service With Celebrity Parade but Few Details
4,"Apple Cracks Down on Apps That Fight iPhone Addiction, Last year, with much fanfare, the tech giant unveiled a screen-time tracker of its own. Then it quietly began purging competitors from its store.",Technology,"Last year, with much fanfare, the tech giant unveiled a screen-time tracker of its own. Then it quietly began purging competitors from its store.",2019-04-27,Apple Cracks Down on Apps That Fight iPhone Addiction


### 2. Stock price history from Yahoo Finance

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#### 2.1. Obtain stock price data from Yahoo Finance:

Obtain Apple stock (AAPL)'s daily Open,High,Low,Close price,Volume data from period 2010-01-01 to 2019-04-30. Control web browser by Selenium and collect data by Beautifulsoup.

In [7]:
page = 'https://finance.yahoo.com/quote/AAPL/history?period1=1262275200&period2=1556640000&interval=1d&filter=history&frequency=1d'
browser.get(page)
headers = {'User-Agent':'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/534.14 (KHTML, like Gecko) Chrome/9.0.600.0 Safari/534.14'}
r = requests.get(page,headers=headers)

In [1106]:
# scroll down the page iteratively with a delay
for _ in range(0, 25):
    browser.execute_script("window.scrollTo(0, 150000);")
    time.sleep(5)

In [1107]:
r = browser.page_source

In [1172]:
soup = BeautifulSoup(r)

In [1185]:
# Extract the relevant information as text
# Find the element on the webpage
main_content = soup.find('table', attrs = {'data-test': 'historical-prices'})
content = main_content.find_all('tr')
content = [x.getText(separator=u'/') for x in content]

In [1233]:
apple_df = pd.DataFrame([each.split('/')for each in content][1:-1], columns=[each.split('/')for each in content][0])
apple_df.head()

,Date,Open,High,Low,Close*,Adj Close**,Volume
0,"Apr 30, 2019",203.06,203.40,199.11,200.67,199.90,"46,534,900"
1,"Apr 29, 2019",204.40,205.97,203.86,204.61,203.83,"22,204,700"
2,"Apr 26, 2019",204.90,205.00,202.12,204.30,203.52,"18,649,100"
3,"Apr 25, 2019",206.83,207.76,205.12,205.28,204.49,"18,543,200"
4,"Apr 24, 2019",207.36,208.48,207.05,207.16,206.37,"17,540,600"


#### 2.2. Data Cleansing:

Drop null values and sort dataframe from old to recent.

In [1234]:
apple_df.dropna(how = 'any',inplace=True)

In [1238]:
reversed_order = apple_df.iloc[::-1]
reversed_order = reversed_order[1:]
reversed_order.head()

,Date,Open,High,Low,Close*,Adj Close**,Volume
2373,"Jan 04, 2010",30.49,30.64,30.34,30.57,20.31,"123,432,400"
2372,"Jan 05, 2010",30.66,30.80,30.46,30.63,20.34,"150,476,200"
2371,"Jan 06, 2010",30.63,30.75,30.11,30.14,20.02,"138,040,000"
2370,"Jan 07, 2010",30.25,30.29,29.86,30.08,19.98,"119,282,800"
2369,"Jan 08, 2010",30.04,30.29,29.87,30.28,20.12,"111,902,700"


Adding more dimension to existing dataframe: Obtain stock return colume to gauge daily price movement.

In [1244]:
apple_px = reversed_order[['Date','Close*']]
#apple_px = pd.DataFrame([float(each) for each in apple_px['Close*']],columns = ['Price Change %'])
apple_px['Converted Px'] = [float(each) for each in apple_px['Close*']]
apple_px['Px change %'] = apple_px['Converted Px'].pct_change(1)

C:\Users\Natalie Kim\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Natalie Kim\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [1250]:
merged_df = reversed_order.merge(apple_px[['Date','Px change %']], on='Date')
merged_df.head()

,Date,Open,High,Low,Close*,Adj Close**,Volume,Px change %
0,"Jan 04, 2010",30.49,30.64,30.34,30.57,20.31,"123,432,400",NaN
1,"Jan 05, 2010",30.66,30.80,30.46,30.63,20.34,"150,476,200",0.001963
2,"Jan 06, 2010",30.63,30.75,30.11,30.14,20.02,"138,040,000",-0.015997
3,"Jan 07, 2010",30.25,30.29,29.86,30.08,19.98,"119,282,800",-0.001991
4,"Jan 08, 2010",30.04,30.29,29.87,30.28,20.12,"111,902,700",0.006649


Convert Date format to match NY Time Date format for consistency.

In [1251]:
converted_date = []
for date in merged_df['Date']:
    dateTime = datetime.datetime.strptime(date, "%b %d, %Y")
    converted_date.append(dateTime.strftime('%d/%m/%Y'))

In [1253]:
merged_df['Date'] = converted_date

In [1255]:
save_as_csv(merged_df,'Yahoo_data') #for backup

#### 2.3. Merge NYT and Price dataframe:

This dataframe will be used in fundamental analysis. In fundamental analysis, text analytics is performed to see if positive or negative news have impacted stock price direction.

In [1264]:
news_and_px = pd.merge(sub_df, merged_df,left_on='CONVERTED_DATE',right_on='Date',how='inner')
direction = []
for each in news_and_px['Px change %']:
    if each >= 0:
        direction.append(1) #price move up or stay =1
    else:
        direction.append(0) #price move down =0
news_and_px['Direction'] = direction
news_and_px.head()

,ALL_CONTENT,CATEGORIES,CONTENT,DATE,TITLE,CONVERTED_DATE,Date,Open,High,Low,Close*,Adj Close**,Volume,Px change %,Direction
0,"Apple AirPods Review: Perfect Earbuds, but They Don’t Last, Sleek and convenient, Apple’s wireless earbuds are an engineering marvel. One problem: They are difficult and expensive to maintain.",Personal Tech,"Sleek and convenient, Apple’s wireless earbuds are an engineering marvel. One problem: They are difficult and expensive to maintain.","April 3, 2019","Apple AirPods Review: Perfect Earbuds, but They Don’t Last",03/04/2019,03/04/2019,193.25,196.50,193.15,195.35,194.60,"23,271,800",0.006855,1
1,"Apple Unveils Video Service With Celebrity Parade but Few Details, The iPhone maker is partnering with Hollywood heavyweights and publishers to provide broad new services to its users.",Technology,The iPhone maker is partnering with Hollywood heavyweights and publishers to provide broad new services to its users.,"March 26, 2019",Apple Unveils Video Service With Celebrity Parade but Few Details,26/03/2019,26/03/2019,191.66,192.88,184.58,186.79,186.07,"49,800,500",-0.010332,0
2,"Samsung Predicts Disappointing Results as Chip Prices Fall, Weaker demand for smartphones in China and elsewhere has put increasing pressure on the gadget industry.",Business,Weaker demand for smartphones in China and elsewhere has put increasing pressure on the gadget industry.,"March 26, 2019",Samsung Predicts Disappointing Results as Chip Prices Fall,26/03/2019,26/03/2019,191.66,192.88,184.58,186.79,186.07,"49,800,500",-0.010332,0
3,"Apple and Qualcomm Settle All Disputes Worldwide, Apple and Qualcomm said they had agreed to dismiss all litigation between them worldwide.",Technology,Apple and Qualcomm said they had agreed to dismiss all litigation between them worldwide.,"April 17, 2019",Apple and Qualcomm Settle All Disputes Worldwide,17/04/2019,17/04/2019,199.54,203.38,198.61,203.13,202.35,"28,906,800",0.019473,1
4,"Organizing Your Unwieldy Photo Collection Is Easier Than You Think, Here is the trick: Let your smartphone do the work.",Personal Tech,Here is the trick: Let your smartphone do the work.,"April 10, 2019",Organizing Your Unwieldy Photo Collection Is Easier Than You Think,10/04/2019,10/04/2019,198.68,200.74,198.18,200.62,199.85,"21,695,300",0.005614,1


In [1265]:
save_as_csv(news_and_px,'Fundamental Analysis') #backup